In [45]:
from openpyxl import load_workbook

# Load the workbook
workbook = load_workbook("sam.xlsx")

# Iterate through each sheet in the workbook
for sheet in workbook:
    # Find the index of the 'RT' column
    for col_idx in range(1, sheet.max_column + 1):
        if sheet.cell(row=1, column=col_idx).value == 'RT':
            rt_column_index = col_idx
            print(col_idx)
            break

    # If 'RT' column found
    if rt_column_index:
        # Collect unique values from the 'RT' column
        unique_values = set(sheet.cell(row=row_idx, column=rt_column_index).value
                             for row_idx in range(2, sheet.max_row + 1))

        # Create a list to store sub-tables
        sub_tables = []
        # Create sub-tables for each unique 'RT' value
        for rt_value in unique_values:
            sub_table = [['' for _ in range(sheet.max_column)] for _ in range(sheet.max_row)]  # Initialize sub-table
            
            # Copy headers from the main table
            for col_idx in range(1, sheet.max_column + 1):
                sub_table[0][col_idx - 1] = sheet.cell(row=1, column=col_idx).value

            # Copy rows matching 'RT' value to sub-table
            sub_table_row_idx = 1
            for row_idx in range(2, sheet.max_row + 1):
                if sheet.cell(row=row_idx, column=rt_column_index).value == rt_value:
                    for col_idx in range(1, sheet.max_column + 1):
                        sub_table[sub_table_row_idx][col_idx - 1] = sheet.cell(row=row_idx, column=col_idx).value
                    sub_table_row_idx += 1

            # Add sub-table to the list
            sub_tables.append(sub_table)

        # Clear the original sheet
        sheet.delete_rows(1, sheet.max_row)

        # Add sub-tables to the sheet
        for sub_table in sub_tables:
            for sub_row in sub_table:
                sheet.append(sub_row)
            

# Save the modified workbook
workbook.save("modified_workbook.xlsx")


4
4


In [110]:
### ABSOLUTELY CORRECT

from openpyxl import load_workbook

# Load the workbook
workbook = load_workbook("sam.xlsx")

# Iterate through each sheet in the workbook
for sheet in workbook:
    # Find the index of the 'RT' column
    rt_column_index = None
    for col_idx in range(1, sheet.max_column + 1):
        if sheet.cell(row=1, column=col_idx).value == 'RT':
            rt_column_index = col_idx
            break

    # If 'RT' column found
    if rt_column_index:
        # Collect unique values from the 'RT' column
        unique_values = set(sheet.cell(row=row_idx, column=rt_column_index).value
                             for row_idx in range(2, sheet.max_row + 1))

        # Create a list to store sub-tables
        sub_tables = []

        # Create sub-tables for each unique 'RT' value
        for rt_value in unique_values:
            sub_table = []  # Initialize sub-table

            # Add heading for the sub-table
            sub_table.append([f"Sub-table for '{rt_value}'"])

            # Copy headers from the main table
            sub_table.append([sheet.cell(row=1, column=col_idx).value for col_idx in range(1, sheet.max_column + 1)])

            # Copy rows matching 'RT' value to sub-table
            for row_idx in range(2, sheet.max_row + 1):
                if sheet.cell(row=row_idx, column=rt_column_index).value == rt_value:
                    sub_table.append([sheet.cell(row=row_idx, column=col_idx).value for col_idx in range(1, sheet.max_column + 1)])

            # Add sub-table to the list
            sub_tables.append(sub_table)

        # Clear the original sheet
        sheet.delete_rows(1, sheet.max_row)

        # Add sub-tables to the sheet
        for sub_table in sub_tables:
            for sub_row in sub_table:
                sheet.append(sub_row)  # Append each row of the sub-table
            sheet.append([])  # Add a gap of one row after each sub-table

# Save the modified workbook
workbook.save("modified_workbook.xlsx")


In [80]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel("your_workbook.xlsx")

# Group records by 'RT', 'Country', and 'City'
grouped = df.groupby(['RT', 'Country', 'City'])

# Create a new DataFrame to store the results
result = pd.DataFrame(columns=['GCB', 'Country', 'City', 'Justify', 'Proposal'])

# Iterate through each group
for name, group in grouped:
    # Group records by 'GCB' value and count the occurrences
    gcb_counts = group['GCB'].value_counts()
    
    # Iterate through each 'GCB' value and count
    for gcb, count in gcb_counts.items():
        # Concatenate 'Justify' values
        justify = ', '.join(group['justify'])
        
        # Create a new row with the aggregated values
        new_row = {
            'GCB': f"{count}*{gcb}",
            'Country': name[1],
            'City': name[2],
            'Justify': justify,
            'Proposal': ''  # Initialize 'Proposal' column with empty string
        }
        
        # Append the new row to the result DataFrame
        result = result.append(new_row, ignore_index=True)

# Save the result to a new Excel file
result.to_excel("result.xlsx", index=False)


Table_Sheet1_0 in sheet Sheet1
Table_Sheet1_1 in sheet Sheet1
Table_Sheet1_2 in sheet Sheet1
Table_Sheet2_0 in sheet Sheet2
Table_Sheet2_1 in sheet Sheet2
Table_Sheet2_2 in sheet Sheet2


In [118]:
## good for borders
from openpyxl import load_workbook
from openpyxl.styles import Border, Side

# Load the workbook
workbook = load_workbook("modified_workbook.xlsx")

# Access a specific sheet
sheet = workbook["Sheet1"]

# Define border styles
border_style = Border(left=Side(style='thin'),
                      right=Side(style='thin'),
                      top=Side(style='thin'),
                      bottom=Side(style='thin'))

# Iterate through each row in the sheet
for row in sheet.iter_rows():
    # Check if the row has more than one cell with a value
    if sum(1 for cell in row if cell.value) > 1:
        # Iterate through each cell in the row
        for cell in row:
            # Apply the border style to the cell
            cell.border = border_style

# Save the modified workbook
workbook.save("modified_workbook1.xlsx")


In [90]:
no formatting is applied


SyntaxError: invalid syntax (<ipython-input-90-2796c81f9b12>, line 1)